In [3]:
"""
2025-11-05_HrConfirmationACTG320

Purpose
-------
Replicate Hammer et al. (1997) treatment hazard ratios (overall and by CD4 strata)
using the ACTG320 dataset variant from scikit-survival, then fit a multivariable
CoxPH using the six covariates used in your appendix:
  Age, Sex, CD4 stratum (≤50 vs 51–200), Treatment, Functional Impairment (KPS-derived),
  Months of prior ZDV use.
"""

!pip install -q scikit-survival lifelines

# --- Imports -----------------------------------------------------------------
import numpy as np
import pandas as pd
from lifelines import CoxPHFitter
import random, os

# scikit-survival data
from sksurv.datasets import load_aids

# --- Reproducibility ----------------------------------------------------------
def seed_everything(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

seed_everything(42)

# --- Load ACTG320 / AIDS dataset ---------------------------------------------
# sksurv returns: X as a pandas DataFrame, y as a structured array with fields (event, time)
# For AIDS, the fields are named ('censor', 'time') in older versions, with 'censor' often being the event flag.
# We'll coerce a clean (Duration, Event) pair where Event is 1 if event occurred.
X, y_sa = load_aids()

# Convert structured array -> DataFrame
y = pd.DataFrame(y_sa)
# Harmonise column names to ('Duration','Event') with Event in {0,1}
if "time" in y.columns:
    y.rename(columns={"time": "Duration"}, inplace=True)
if "censor" in y.columns:
    # Many sksurv loaders store boolean-like flags; make them 0/1 ints.
    y["Event"] = y["censor"].astype(int)
    y.drop(columns=["censor"], inplace=True)
elif "event" in y.columns:
    y["Event"] = y["event"].astype(int)
    y.drop(columns=["event"], inplace=True)

df = pd.concat([X.reset_index(drop=True), y.reset_index(drop=True)], axis=1)

# --- Variable engineering (aligned to your DataRecords script) ----------------
# Rename to readable labels (match your prior convention)
df = df.rename(columns={
    "age": "Age",
    "cd4": "CD4_Count",
    "hemophil": "Hemophilia",
    "ivdrug": "IV_Drug_Use",
    "karnof": "Karnofsky_Score",
    "priorzdv": "Months_Prior_ZDV",
    "raceth": "Race_Ethnicity",
    "sex": "Sex",
    "strat2": "CD4_Stratification",
    "tx": "Treatment_Indicator",
})

# Primary CD4 split feature
df["CD4_51_200"] = df["CD4_Count"].apply(lambda v: 1 if (51 <= v <= 200) else 0)

# Drop the original CD4 count for modelling (we use the split)
df = df.drop(columns=["CD4_Count", "CD4_Stratification"], errors="ignore")

# Karnofsky (categorical 100,90,80,70) -> Severity 0..3 (higher = worse)
df["Karnofsky_Score"] = df["Karnofsky_Score"].astype(int)
df["KScale_Severity"] = df["Karnofsky_Score"].replace({100: 0, 90: 1, 80: 2, 70: 3})
df = df.drop(columns=["Karnofsky_Score"])

# Coerce core inputs to int
for col in ["Race_Ethnicity", "IV_Drug_Use", "Treatment_Indicator", "Sex", "Hemophilia"]:
    if col in df.columns:
        df[col] = df[col].astype(int)

# In the original ACTG320 mapping: Sex {0: Female, 1: Male}
# Some sksurv versions use 1/2; enforce 0/1 with 0=Female, 1=Male
# (If already 0/1, this is a no-op)
df["Sex"] = df["Sex"].replace({2: 0, 1: 1}).astype(int)

# Keep the minimal set as in my appendix/model:
cols_core = [
    "CD4_51_200",           # 1 if 51–200, else 0 (≤50)
    "Treatment_Indicator",  # 1 if triple-therapy, 0 if double
    "KScale_Severity",      # 0..3 from KPS
    "Age",                  # numeric
    "Sex",                  # 0=F, 1=M
    "Months_Prior_ZDV",     # numeric
    "Duration", "Event"
]
df = df[cols_core].copy()

# Sanity: coerce event to int 0/1
df["Event"] = df["Event"].astype(int)

# --- Pretty labels for outputs ------------------------------------------------
pretty = {
    "CD4_51_200": "CD4 51–200",
    "Treatment_Indicator": "Treatment (3-drug vs 2-drug)",
    "KScale_Severity": "Functional Impairment (KPS→severity)",
    "Age": "Age (years)",
    "Sex": "Sex (1=Male)",
    "Months_Prior_ZDV": "Months Prior ZDV",
    "Duration": "Duration",
    "Event": "Event"
}

# --- Helper: run CoxPH and print a NEJM-style table ---------------------------
from lifelines import CoxPHFitter

def cox_univariate_hr(df_in: pd.DataFrame, duration="Duration", event="Event", var="Treatment_Indicator"):
    cph = CoxPHFitter()
    cph.fit(df_in[[var, duration, event]], duration_col=duration, event_col=event)
    summ = cph.summary.loc[var]
    hr = summ["exp(coef)"]
    l = summ["exp(coef) lower 95%"]
    u = summ["exp(coef) upper 95%"]
    return hr, l, u

def print_neat(title, rows, headers=("Group", "HR (95% CI)")):
    print("\n" + title)
    print("-" * len(title))
    w1 = max(len(r[0]) for r in rows) + 2
    print(f"{headers[0]:<{w1}}{headers[1]}")
    for g, txt in rows:
        print(f"{g:<{w1}}{txt}")

def fmt_hr(hr, l, u):
    return f"{hr:.2f} ({l:.2f}, {u:.2f})"

# --- (A) Hammer et al. (1997) replication: treatment effect -------------------
# Overall
hr_all = cox_univariate_hr(df, var="Treatment_Indicator")
# CD4 ≤50
df_le50 = df[df["CD4_51_200"] == 0]
hr_le50 = cox_univariate_hr(df_le50, var="Treatment_Indicator")
# CD4 51–200
df_51_200 = df[df["CD4_51_200"] == 1]
hr_51_200 = cox_univariate_hr(df_51_200, var="Treatment_Indicator")

rows = [
    ("All patients",        fmt_hr(*hr_all)),
    ("CD4 ≤50 cells/mm³",   fmt_hr(*hr_le50)),
    ("CD4 51–200 cells/mm³",fmt_hr(*hr_51_200)),
]
print_neat(
    "Table A. Hammer et al. replication — Treatment Hazard Ratios (univariate Cox)",
    rows
)

# --- (B) Multivariable CoxPH with the six covariates --------------------------
df_mv = df.rename(columns=pretty)
covariates = [
    "Age (years)",
    "Sex (1=Male)",
    "CD4 51–200",
    "Treatment (3-drug vs 2-drug)",
    "Functional Impairment (KPS→severity)",
    "Months Prior ZDV",
]
cph_mv = CoxPHFitter()
cph_mv.fit(df_mv[covariates + ["Duration", "Event"]], duration_col="Duration", event_col="Event")

mv = cph_mv.summary.loc[covariates, ["exp(coef)", "exp(coef) lower 95%", "exp(coef) upper 95%"]].round(2)
print("\nTable B. Multivariable CoxPH — Six Covariates (HR with 95% CI)")
print(mv.to_string())



Table A. Hammer et al. replication — Treatment Hazard Ratios (univariate Cox)
-----------------------------------------------------------------------------
Group                 HR (95% CI)
All patients          0.50 (0.33, 0.77)
CD4 ≤50 cells/mm³     0.58 (0.36, 0.95)
CD4 51–200 cells/mm³  0.38 (0.17, 0.88)

Table B. Multivariable CoxPH — Six Covariates (HR with 95% CI)
                                      exp(coef)  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                
Age (years)                                1.02                 1.00                 1.04
Sex (1=Male)                               0.93                 0.53                 1.62
CD4 51–200                                 0.41                 0.26                 0.64
Treatment (3-drug vs 2-drug)               0.50                 0.33                 0.77
Functional Impairment (KPS→severity)       1.93                 1.53                 